In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from scipy.stats import norm

In [2]:
# pd.set_option('display.float_format', lambda x: '%.f' % x)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

### 1. Problem Definition

To predict stock price appreciation based on financial data (fundamental approach) for Indonesian public companies from 2020 until 2023.

### 2. Data Collection

In [3]:
df = pd.read_csv('combined_financial_data_idx.csv')

In [4]:
df.head()

,symbol,account,type,2020,2021,2022,2023
0,AALI,Accounts Payable,BS,7.702640e+11,1.026717e+12,1.224423e+12,8.420640e+11
1,AALI,Accounts Receivable,BS,7.658490e+11,4.581350e+11,8.487700e+11,6.744870e+11
2,AALI,Accumulated Depreciation,BS,-1.092095e+13,-1.213381e+13,-1.330375e+13,-1.443685e+13
3,AALI,Additional Paid In Capital,BS,3.878995e+12,3.878995e+12,3.878995e+12,3.878995e+12
4,AALI,Allowance For Doubtful Accounts Receivable,BS,-2.426100e+10,-2.454300e+10,-2.705700e+10,-2.651600e+10


In [5]:
df = df.pivot(index='symbol', columns='account', values=['2020', '2021', '2022', '2023'])
df.columns = [f'{col[1]}_{col[0]}' for col in df.columns]
df.reset_index(inplace=True)

In [6]:
df.head()

,symbol,Accounts Payable_2020,Accounts Receivable_2020,Accumulated Depreciation_2020,Additional Paid In Capital_2020,Allowance For Doubtful Accounts Receivable_2020,Amortization_2020,Assets Held For Sale Current_2020,Available For Sale Securities_2020,Average Dilution Earnings_2020,...,Total Tax Payable_2023,Total Unusual Items_2023,Total Unusual Items Excluding Goodwill_2023,Tradeand Other Payables Non Current_2023,Trading Securities_2023,Treasury Shares Number_2023,Treasury Stock_2023,Work In Process_2023,Working Capital_2023,Write Off_2023
0,AALI,7.702640e+11,7.658490e+11,-1.092095e+13,3.878995e+12,-2.426100e+10,NaN,NaN,NaN,NaN,...,1.202370e+11,7.490100e+10,7.490100e+10,NaN,NaN,0.0,NaN,1.099140e+11,3.236061e+12,-2.384100e+10
1,ABBA,3.835642e+10,1.589415e+10,-1.764548e+11,-1.012454e+11,-6.841390e+10,NaN,NaN,8.225640e+10,NaN,...,1.718310e+10,NaN,NaN,1.026090e+11,NaN,NaN,NaN,NaN,-1.025919e+11,NaN
2,ABDA,7.686395e+09,8.013687e+10,-9.088003e+10,8.109426e+09,NaN,NaN,NaN,NaN,NaN,...,3.849437e+09,8.119158e+09,8.119158e+09,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
3,ABMM,1.156274e+08,1.386149e+08,-6.859381e+08,1.150872e+08,-5.242683e+07,NaN,0.0,2.002887e+07,NaN,...,9.567298e+06,NaN,NaN,NaN,NaN,0.0,NaN,1.039217e+07,-8.958664e+06,NaN
4,ACES,1.642272e+11,1.434827e+11,-8.396180e+11,4.405749e+11,-5.511007e+07,3.706185e+09,NaN,NaN,NaN,...,7.371071e+10,-5.739006e+08,-5.739006e+08,NaN,NaN,29610300.0,3.418487e+10,NaN,4.898755e+12,5.463870e+07


In [7]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Accounts Payable_2020,581.0,4.966911e+11,1.706642e+12,0.000000e+00,6.615148e+09,8.067720e+10,3.300470e+11,1.699900e+13
Accounts Receivable_2020,530.0,5.727652e+11,1.676043e+12,0.000000e+00,4.702138e+09,8.769114e+10,4.007902e+11,1.707864e+13
Accumulated Depreciation_2020,588.0,-2.057908e+12,1.001841e+13,-1.664430e+14,-1.039924e+12,-1.984147e+11,-2.376867e+10,0.000000e+00
Additional Paid In Capital_2020,577.0,8.624769e+11,3.342041e+12,-4.111710e+12,4.570000e+08,8.167701e+10,5.698250e+11,6.328082e+13
Allowance For Doubtful Accounts Receivable_2020,405.0,-8.417233e+10,4.783591e+11,-8.360000e+12,-3.574800e+10,-3.776131e+09,-1.267331e+08,0.000000e+00
...,...,...,...,...,...,...,...,...
Treasury Shares Number_2023,216.0,7.915106e+08,8.925468e+09,0.000000e+00,0.000000e+00,1.045300e+04,1.054315e+08,1.308512e+11
Treasury Stock_2023,116.0,3.022673e+11,9.796827e+11,0.000000e+00,7.526860e+08,2.250151e+10,1.221651e+11,8.199511e+12
Work In Process_2023,143.0,9.994955e+10,2.273102e+11,0.000000e+00,1.640456e+09,1.576900e+10,8.045000e+10,1.836794e+12
Working Capital_2023,457.0,1.071514e+12,4.735861e+12,-1.987207e+13,8.225521e+06,1.602316e+11,9.218260e+11,4.116400e+13


### 3. Data Preprocessing

#### 3.1. Missing Values

In [8]:
df[df.isnull().any(axis=1)].head(5)

,symbol,Accounts Payable_2020,Accounts Receivable_2020,Accumulated Depreciation_2020,Additional Paid In Capital_2020,Allowance For Doubtful Accounts Receivable_2020,Amortization_2020,Assets Held For Sale Current_2020,Available For Sale Securities_2020,Average Dilution Earnings_2020,...,Total Tax Payable_2023,Total Unusual Items_2023,Total Unusual Items Excluding Goodwill_2023,Tradeand Other Payables Non Current_2023,Trading Securities_2023,Treasury Shares Number_2023,Treasury Stock_2023,Work In Process_2023,Working Capital_2023,Write Off_2023
0,AALI,7.702640e+11,7.658490e+11,-1.092095e+13,3.878995e+12,-2.426100e+10,NaN,NaN,NaN,NaN,...,1.202370e+11,7.490100e+10,7.490100e+10,NaN,NaN,0.0,NaN,1.099140e+11,3.236061e+12,-2.384100e+10
1,ABBA,3.835642e+10,1.589415e+10,-1.764548e+11,-1.012454e+11,-6.841390e+10,NaN,NaN,8.225640e+10,NaN,...,1.718310e+10,NaN,NaN,1.026090e+11,NaN,NaN,NaN,NaN,-1.025919e+11,NaN
2,ABDA,7.686395e+09,8.013687e+10,-9.088003e+10,8.109426e+09,NaN,NaN,NaN,NaN,NaN,...,3.849437e+09,8.119158e+09,8.119158e+09,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
3,ABMM,1.156274e+08,1.386149e+08,-6.859381e+08,1.150872e+08,-5.242683e+07,NaN,0.0,2.002887e+07,NaN,...,9.567298e+06,NaN,NaN,NaN,NaN,0.0,NaN,1.039217e+07,-8.958664e+06,NaN
4,ACES,1.642272e+11,1.434827e+11,-8.396180e+11,4.405749e+11,-5.511007e+07,3.706185e+09,NaN,NaN,NaN,...,7.371071e+10,-5.739006e+08,-5.739006e+08,NaN,NaN,29610300.0,3.418487e+10,NaN,4.898755e+12,5.463870e+07


In [9]:
# Imputing null values with zero
df = df.fillna(0)

#### 3.2. Skewed Distribution

In [10]:
# WIP

#### 3.3. Outliers

In [11]:
# WIP

### 4. Exploratory Data Analysis

### 4.1. Target Variables

In [12]:
# WIP

# Future stock price:
# 1 April 2021 - 1 April 2022; to predict the effect of AR 2021 (fundamental) on stock price
# 1 April 2022 - 1 April 2023; to predict the effect of AR 2022 (fundamental) on stock price
# 1 April 2023 - 1 April 2024; to predict the effect of AR 2023 (fundamental) on stock price

# Future dividends

# Future earnings

### 4.2. Predictor Variables

In [13]:
# WIP

#### 4.2.1. Return on Equity (ROE)

In [14]:
years = [2020, 2021, 2022, 2023]

for year in years:
    net_income_col = f"Net Income_{year}"
    total_equity_col = f"Total Equity Gross Minority Interest_{year}"
    new_col = f"ROA_{year}"
    
    df[new_col] = df[net_income_col] / df[total_equity_col]

#### 4.2.2. Retention Ratio (RR)

In [15]:
for year in years:
    dividend_col = f"Cash Dividends Paid_{year}"
    net_income_col = f"Net Income_{year}"
    new_col = f"RR_{year}"
    
    df[new_col] = df[dividend_col] / df[net_income_col]

#### 4.2.3. Sustainability Growth Rate (SGR)

In [16]:
for year in years:
    ROA_col = f"ROA_{year}"
    RR_col = f"RR_{year}"
    new_col = f"SGR_{year}"
    
    df[new_col] = df[ROA_col] * df[RR_col]

### 5. Feature Selection

In [17]:
variables = ["symbol",
             "Net Income_2023",
             "SGR_2020", "SGR_2021", "SGR_2022"]

df = df.drop(columns=[col for col in df.columns if col not in variables])

In [18]:
df.head()

,symbol,Net Income_2023,SGR_2020,SGR_2021,SGR_2022
0,AALI,1.055897e+12,-0.009099,-0.023180,-0.038415
1,ABBA,-4.370504e+10,-0.000000,0.000000,0.000000
2,ABDA,8.458131e+10,-0.026851,-0.027507,-0.033610
3,ABMM,2.890006e+08,-0.015597,0.000000,-0.107696
4,ACES,7.635075e+11,-0.057880,-0.098660,-0.059405


### 5.1. Target Variables

In [19]:
y = df["Net Income_2023"]

### 5.2. Predictor Variables

In [20]:
X = df[["SGR_2020", "SGR_2021", "SGR_2022"]]

In [25]:
df[df.isnull().any(axis=1)].head(5)

,symbol,Net Income_2023,SGR_2020,SGR_2021,SGR_2022
40,ARMY,0.0,0.0,NaN,NaN
77,BDKR,0.0,NaN,0.0,0.0
189,DOOH,427796101.0,NaN,0.0,0.0
224,GAMA,0.0,0.0,0.0,NaN
234,GOLL,0.0,0.0,NaN,NaN


In [28]:
# Imputing null values with zero
X = X.fillna(0)

### 6. Data Modelling

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge

In [30]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
# Ridge regression with cross-validation
ridge_model = Ridge()
ridge_scores = cross_val_score(ridge_model, X, y, cv=5, scoring='r2')

print(f"Cross-Validated Ridge R^2 Scores: {ridge_scores}")
print(f"Mean Cross-Validated Ridge R^2 Score: {ridge_scores.mean()}")

Cross-Validated Ridge R^2 Scores: [-0.04774876 -0.02870421  0.06579858 -1.83224406  0.03376325]
Mean Cross-Validated Ridge R^2 Score: -0.36182703880617584
